### **Data Preprocessing**

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics

import os


In [ ]:
# Useful Constants

# Those are separate normalised input features for the neural network
INPUT_SIGNAL_TYPES = [
    "x_accel",
    "y_accel",
    "z_accel",
]

# Output classes to learn how to classify
LABELS = [
    "Walking", 
    "Jogging", 
    "Uptairs", 
    "Downstairs", 
    "Sitting", 
    "Standing"
] 

In [ ]:
from numpy import genfromtxt
import pandas as pd
x = genfromtxt('/content/drive/MyDrive/Colab Notebooks/WISDM_x.csv', delimiter=',')
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/WISDM_y.csv').to_numpy()

In [ ]:
def create_series(x,y,timestep,overlap):

  slide_step = int(timestep*(1-overlap))
  data_num = int((len(x)/slide_step)-1)
  
  dataset = np.ndarray(shape=(data_num,timestep,len(x[0])))
  labels = list()

  for i in range(data_num):
    labels.append(y[slide_step*(i+1)-1])
    for j in range(timestep):
      dataset[i,j,:] = x[slide_step*i+j,:]

  return dataset,np.array(labels)


In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le = LabelEncoder()
y = le.fit_transform(y).reshape(-1,1)


ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y[1]
# y2 = y.extend

array([0., 1., 0., 0., 0., 0.])

In [ ]:
# y_pred.append("5")
dataset,labels = create_series(x,y,256,0.5)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset,labels,test_size=0.3, random_state=412)
# X_test, X_val, y_test, y_val = train_test_split(X_toSplit,y_toSplit,test_size=0.25, random_state=412)

In [ ]:
# print(X_train.shape,
# X_test.shape,
# X_val.shape,
# dataset.shape
# )

In [ ]:
labels.shape

(8190, 6)

In [ ]:
# Input Data 

training_data_count = len(X_train)  # 96092 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 41183 testing series
n_steps = len(X_train[0])  # 16 timesteps per series
n_input = len(X_train[0][0])  # 3 input parameters per timestep


# LSTM Neural Network's internal structure
# n_hidden = 32 # Hidden layer num of features
n_classes = 6 # Total classes (should go up, or should go down)


# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 64
display_iter = 30000  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test), np.std(X_test))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)
(2457, 256, 3) (2457, 6) 2.882226147169298 6.897425254528976
The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.


### **Model**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.LSTM(256, return_sequences=True,input_shape=(n_steps, n_input)))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(64, return_sequences=True))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(32))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(6, activation='softmax'))

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])

history = model.fit(X_train,y_train,batch_size=64,epochs=10000,validation_split=0.1,callbacks=[callback])

model.save('/Users/wangshiyu/Desktop/lstm_model')

Epoch 1/10000
81/81 [==============================] - 11s 72ms/step - loss: 0.9979 - accuracy: 0.6649 - val_loss: 0.9254 - val_accuracy: 0.7213
Epoch 2/10000
81/81 [==============================] - 4s 54ms/step - loss: 0.8154 - accuracy: 0.7089 - val_loss: 0.8477 - val_accuracy: 0.7352
Epoch 3/10000
81/81 [==============================] - 4s 54ms/step - loss: 0.9702 - accuracy: 0.6792 - val_loss: 0.7298 - val_accuracy: 0.7509
Epoch 4/10000
81/81 [==============================] - 4s 54ms/step - loss: 0.7562 - accuracy: 0.7327 - val_loss: 0.6722 - val_accuracy: 0.7456
Epoch 5/10000
81/81 [==============================] - 4s 54ms/step - loss: 0.6259 - accuracy: 0.7775 - val_loss: 0.5434 - val_accuracy: 0.8014
Epoch 6/10000
81/81 [==============================] - 5s 59ms/step - loss: 0.6231 - accuracy: 0.7790 - val_loss: 0.5295 - val_accuracy: 0.8136
Epoch 7/10000
81/81 [==============================] - 4s 54ms/step - loss: 0.5887 - accuracy: 0.7953 - val_loss: 0.5154 - val_accuracy

In [ ]:
model.evaluate(X_test, y_test)

77/77 [==============================] - 2s 22ms/step - loss: 0.1227 - accuracy: 0.9666


[0.1227295771241188, 0.966625988483429]